# 영화 추천시스템

### 라이브러리 및 데이터 불러오기

In [1]:
import pandas as pd
from numpy import *
import numpy as np
import matplotlib.pyplot as plt

In [19]:
ratings = pd.read_csv("ratings_small.csv")
# print (ratings.head())

In [18]:
movies = pd.read_csv("movies.csv")
# print (movies.head())

###  영화 평점과 이름 병합 

In [4]:
ratings = pd.merge(ratings[['userId','movieId','rating']],
                   movies[['movieId','title']], how='left',
                   left_on ='movieId' ,right_on = 'movieId')

In [6]:
# ratings에서 pivot table생성
rp = ratings.pivot_table(columns = ['movieId'],index = ['userId'],values = 'rating')
rp = rp.fillna(0)

In [17]:
# rp.head()

### 판다스 데이터프레임을 넘파이로 변환

In [8]:
rp_mat = mat(rp)

In [16]:
# rp_mat

### 코사인거리 계산

In [10]:
from scipy.spatial.distance import cosine

In [11]:
# 코사인거리 계산 예시
# a= np.asarray( [2, 1, 0, 2, 0, 1, 1, 1])
# b = np.asarray( [2, 1, 1, 1, 1, 0, 1, 1])

# print ("Cosine similarity between A and B is",round(1-cosine(a,b),4))

## 1. 사용자 기반 유사성 매트릭스

In [15]:
# 행(사용자 수)과 열(영화 수)의 개수
m, n = rp.shape
# print(m,n)

In [13]:
# 사용자수 * 사용자수 0행렬 생성
mat_users = np.zeros((m, m))

In [14]:
# mat_users

In [20]:
# 사용자간 코사인 유사성을 구함
for i in range(m):
    for j in range(m):
        if i != j:
            mat_users[i][j] = (1- cosine(rp_mat[i,:], rp_mat[j,:]))
        else:
            mat_users[i][j] = 0.

In [22]:
# 사용자간 유사성 데이터프레임으로 저장
pd_users = pd.DataFrame(mat_users,index =rp.index ,columns= rp.index )
pd_users.head()

userId       1         2         3         4         5         6         7    \
userId                                                                         
1       0.000000  0.000000  0.000000  0.074482  0.016818  0.000000  0.083884   
2       0.000000  0.000000  0.124295  0.118821  0.103646  0.000000  0.212985   
3       0.000000  0.124295  0.000000  0.081640  0.151531  0.060691  0.154714   
4       0.074482  0.118821  0.081640  0.000000  0.130649  0.079648  0.319745   
5       0.016818  0.103646  0.151531  0.130649  0.000000  0.063796  0.095888   

userId       8         9         10   ...       204       205       206  \
userId                                ...                                 
1       0.000000  0.012843  0.000000  ...  0.000000  0.000000  0.000000   
2       0.113190  0.113333  0.043213  ...  0.000000  0.072173  0.021929   
3       0.249781  0.134475  0.114672  ...  0.000000  0.110968  0.051140   
4       0.191013  0.030417  0.137186  ...  0.048892  0.085509  0.252688   
5       0.165712  0.086616  0.032370  ...  0.059153  0.066815  0.055752   

userId       207       208       209       210       211       212       213  
userId                                                                        
1       0.152746  0.000000  0.000000  0.000000  0.025665  0.053411  0.014873  
2       0.038117  0.035502  0.021677  0.071819  0.135890  0.187098  0.134630  
3       0.051725  0.128808  0.000000  0.000000  0.096803  0.164688  0.123897  
4       0.040471  0.083486  0.049957  0.147393  0.063217  0.227245  0.192670  
5       0.024227  0.047905  0.000000  0.074473  0.050070  0.286572  0.174631  

[5 rows x 213 columns]

### 1.1 유사한 사용자 찾기

In [24]:
# uid를 가진 사용자와 가장 유사한 사용자 n명을 찾아내는 함수 
def topn_simusers(uid ,n):
    users = pd_users.loc[uid,:].sort_values(ascending = False)
    topn_users = users.iloc[:n,]
    topn_users = topn_users.rename('score')    
    print ("Similar users as user:",uid)
    return pd.DataFrame(topn_users)

In [25]:
# 17 uid를 가진 유저와 비슷한 사용자 n명 구하기
# print (topn_simusers(uid=17,n=10))

### 2.2 높은 평점 영화 찾기

In [29]:
# uid를 가진 사용자가 높은 평점을 준 영화 n개 찾기
def topn_movieratings(uid ,n_ratings):    
    uid_ratings = ratings.loc[ratings['userId']==uid]
    uid_ratings = uid_ratings.sort_values(by='rating',ascending = [False])
    print ("Top",n_ratings ,"movie ratings of user:",uid)
    return uid_ratings.iloc[:n_ratings,] 

In [30]:
#print (topn_movieratings(uid=196,n_ratings=10))

## 2. 영화 유사성 매트릭스

In [31]:
# 계산 시간 측정 및 영화수*영화수 0행렬 생성
import time
start_time = time.time()
mat_movies = np.zeros((n, n))

In [32]:
#영화 간 유사성 
for i in range(n):
    for j in range(n):
        if i!=j:
            mat_movies[i,j] = (1- cosine(rp_mat[:,i], rp_mat[:,j]))
        else:
            mat_movies[i,j] = 0.
last_time = time.time()

In [33]:
print("--- %s seconds ---" % (last_time - start_time))

--- 1726.5818445682526 seconds ---


In [34]:
pd_movies = pd.DataFrame(mat_movies,index =rp.columns ,columns= rp.columns )

In [35]:
pd_movies.head()

movieId    1         2         3         4         5         6         7       \
movieId                                                                         
1        0.000000  0.395916  0.256876  0.164504  0.189582  0.344532  0.199528   
2        0.395916  0.000000  0.227528  0.326561  0.276072  0.204245  0.265428   
3        0.256876  0.227528  0.000000  0.268179  0.395922  0.204222  0.494572   
4        0.164504  0.326561  0.268179  0.000000  0.244047  0.131439  0.431539   
5        0.189582  0.276072  0.395922  0.244047  0.000000  0.259674  0.438815   

movieId    8         9         10      ...    160080    160271  160438  \
movieId                                ...                               
1        0.112490  0.095801  0.284761  ...  0.056591  0.056591     0.0   
2        0.351015  0.193906  0.425883  ...  0.085907  0.085907     0.0   
3        0.206024  0.382819  0.109113  ...  0.000000  0.000000     0.0   
4        0.691411  0.458831  0.106525  ...  0.000000  0.000000     0.0   
5        0.301315  0.528777  0.223261  ...  0.272853  0.272853     0.0   

movieId    160563    160565    160567    160590    161155    161594    162376  
movieId                                                                        
1        0.056591  0.056591  0.056591  0.113182  0.056591  0.141478  0.141478  
2        0.085907  0.085907  0.085907  0.000000  0.085907  0.107384  0.107384  
3        0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  
4        0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  
5        0.272853  0.272853  0.272853  0.000000  0.272853  0.000000  0.000000  

[5 rows x 5411 columns]

### 2.1 csv파일로 내보내기

In [36]:
pd_movies.to_csv('pd_movies.csv',sep=',')

In [37]:
pd_movies = pd.read_csv("pd_movies.csv",index_col='movieId')

### 2.2 유사한 영화 찾기

In [38]:
# mid를 가진 영화와 비슷한 영화 n개 고르는 함수 생성
import pdb
def topn_simovies(mid ,n):
    mid_ratings = pd_movies.loc[mid,:].sort_values(ascending = False)
    topn_movies = pd.DataFrame(mid_ratings.iloc[:n,])
    pdb.set_trace()
    topn_movies['index1'] = topn_movies.index
    topn_movies['index1'] = topn_movies['index1'].astype('int64')
    topn_movies = pd.merge(topn_movies,movies[['movieId','title']],
                           how = 'left',left_on ='index1' ,right_on = 'movieId')
    print ("Movies similar to movie id:",mid,",",
           movies['title'][movies['movieId']==mid].to_string(index=False),",are")
    del topn_movies['index1']
    return topn_movies

In [39]:
# print (topn_simovies(mid=589,n=15))

## 3. SVD 분석

In [41]:
U,Sigma,VT=linalg.svd(rp_mat)

In [42]:
Sigma

array([288.74592695, 141.2420035 , 120.69848545, 104.5558196 ,
        97.92855509,  93.28582631,  88.19821066,  86.04588056,
        84.07709823,  78.73883217,  73.23711794,  70.53432365,
        69.34166542,  69.07175985,  67.44988983,  66.31942586,
        64.77740082,  64.62758463,  63.63014435,  61.66677188,
        60.28703369,  60.11993174,  59.52959968,  58.64983626,
        57.04081242,  55.66065809,  54.07277667,  53.28740833,
        53.20554049,  52.75730779,  52.42363691,  51.21836761,
        50.8553373 ,  50.15960726,  49.20679687,  48.90618691,
        48.56521915,  48.19529441,  47.74786695,  47.20897421,
        46.94973691,  46.54213668,  45.86137871,  44.983795  ,
        44.66823518,  44.32833769,  44.02781377,  43.27901946,
        43.20762452,  42.91542551,  42.36021609,  42.00372572,
        41.42074972,  41.02842424,  40.74707134,  39.78734835,
        39.38538296,  38.79474677,  38.61773693,  38.49721699,
        38.03524689,  37.94742911,  37.40161675,  37.19

In [43]:
Sig3 = Sigma ** 2
total = sum(Sig3)
total90 = total*.9
print(total, total90)

400803.9999999998 360723.59999999986


In [44]:
sum(Sig3[:105])

361380.69787404046

In [45]:
from numpy import *
from numpy import linalg as la

In [46]:
def ecludSim(inA,inB):
    return 1.0/(1.0 + la.norm(inA - inB))

def pearsSim(inA,inB):
    if len(inA) < 3 : return 1.0
    return 0.5+0.5*corrcoef(inA, inB, rowvar = 0)[0][1]

def cosSim(inA,inB):
    num = float(inA.T*inB)
    denom = la.norm(inA)*la.norm(inB)
    return 0.5+0.5*(num/denom)

In [47]:
def svdEst(dataMat, user, simMeas, item):
    n = shape(dataMat)[1]
    simTotal = 0.0; ratSimTotal = 0.0
    U,Sigma,VT = la.svd(dataMat)
    Sig5 = mat(eye(5)*Sigma[:5]) #arrange Sig5 into a diagonal matrix
    xformedItems = dataMat.T * U[:,:5] * Sig5.I  #create transformed items
    for j in range(n):
        userRating = dataMat[user,j]
        if userRating == 0 or j==item: continue
        similarity = simMeas(xformedItems[item,:].T,\
                             xformedItems[j,:].T)
        #print('the %d and %d similarity is: %f' % (item, j, similarity))
        simTotal += similarity
        ratSimTotal += similarity * userRating
    if simTotal == 0: return 0
    else: return ratSimTotal/simTotal

In [48]:
def recommend(dataMat, user, N=3, simMeas=cosSim, estMethod=svdEst):
    # 1. 점수를 매기지 않은 item 찾기
    unratedItems = nonzero(dataMat[user,:].A==0)[1]
    if len(unratedItems) == 0: return 'you rated everything'
    itemScores = []
    
    # 2. 점수를 매기지 않은 item의 유사도
    for item in unratedItems:
        estimatedScore = estMethod(dataMat, user, simMeas, item)
        itemScores.append((item, estimatedScore))
    
    return sorted(itemScores, key=lambda jj: jj[1], reverse=True)[:N]

In [49]:
# rp_mat을 통해 115 user에게 유사도순에 따라 10개를 보여준다.
recommend(rp_mat,115, N=10, estMethod=svdEst)

KeyboardInterrupt: 